In [ ]:
import pandas as pd
import warnings
import os
from tqdm import tqdm

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Define global variables and paths
classification_file_path = './data/classification.txt'
connections_file_path = './data/connections.txt'
synapses_file_path = './data/synapses.txt'
output_dir = './result/LC11/L2_to_LC11/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Read the data from files
classification = pd.read_csv(classification_file_path)
connections = pd.read_csv(connections_file_path)
synapses = pd.read_csv(synapses_file_path)

# Get the list of L2 and LC11 neurons on the right side
L2_right_ids = classification[(classification['cell_type'] == 'L2') & (classification['side'] == 'right')]['root_id'].tolist()
LC11_right_ids = classification[(classification['hemibrain_type'] == 'LC11') & (classification['side'] == 'right')]['root_id'].tolist()

# Allowed cell types for upstream neurons
allowed_cell_types = {'Pm', 'LC11', 'T4', 'T5', 'Tm', 'Mi', 'Li', 'L2', 'LPi', 'L3', 'L5', 'DM', 'Y'}

# DFS function to find paths and calculate weights
def dfs(neuron_id, visited, connections, current_path, all_paths, current_weight):
    # If the neuron has been visited or the path is too long, return
    if neuron_id in visited or len(current_path) >= 5:
        return
    visited.add(neuron_id)
    current_path.append(neuron_id)

    # Get the upstream neurons connected to the current neuron
    upstream_neurons = connections[connections['post_root_id'] == neuron_id]

    for _, row in upstream_neurons.iterrows():
        pre_id = row['pre_root_id']
        syn_count = row['syn_count']
        nt_type = row['nt_type']

        # Get the cell type of the upstream neuron
        pre_cell_type = classification.loc[classification['root_id'] == pre_id, 'cell_type']
        
        if pre_cell_type.empty:
            continue
        
        # Check if the upstream neuron’s type is allowed
        pre_cell_type_value = pre_cell_type.values[0]  # Get the first value
        
        if not isinstance(pre_cell_type_value, str) or not any(allowed in pre_cell_type_value for allowed in allowed_cell_types):
            continue  # Skip neurons that don't match the allowed types
        
        # Filter synapse types and adjust synapse count
        if nt_type == 'GABA' or nt_type == 'GLUT':
            syn_count = -abs(syn_count)
        elif nt_type == 'ACH':
            syn_count = abs(syn_count)
        else:
            continue
        
        # Get the number of output synapses for the upstream neuron
        output_synapses = synapses.loc[synapses['root_id'] == pre_id, 'output synapses']
        if not output_synapses.empty:
            output_synapses = output_synapses.values[0]
            weight = float(syn_count) / output_synapses
            if abs(weight) < 0.01 and syn_count < 5:
                continue  # Skip paths with negligible weight
            
            # Update the current weight by multiplying with the new weight
            new_weight = current_weight * weight
            
            # Recursively apply DFS for upstream neurons
            dfs(pre_id, visited, connections, current_path, all_paths, new_weight)

    # If the current neuron is an L2 neuron, save the path and its weight
    if neuron_id in L2_right_ids:
        all_paths.append((list(current_path), current_weight))
    
    # Backtrack
    current_path.pop()
    visited.remove(neuron_id)

# Process each LC11 neuron
L2_weights = {L2_id: 0 for L2_id in L2_right_ids}  # Initialize L2 weights
for LC11_neuron in tqdm(LC11_right_ids, desc="Processing LC11 Neurons"):
    visited = set()
    current_path = []
    all_paths = []
    
    # Start DFS for the current LC11 neuron with an initial weight of 1
    dfs(LC11_neuron, visited, connections, current_path, all_paths, 1)
    
    # Update the weight for each L2 neuron based on the paths found
    for path, weight in all_paths:
        L2_neuron_id = path[-1]  # Get the L2 neuron ID from the end of the path
        if L2_neuron_id in L2_weights:
            L2_weights[L2_neuron_id] += weight  # Add the weight to the L2 neuron

    # Write the results to a text file
    output_file_path = os.path.join(output_dir, f'{LC11_neuron}.txt')
    with open(output_file_path, 'w') as f:
        for L2_id, total_weight in L2_weights.items():
            f.write(f'{L2_id}: {total_weight}\n')

    # Reset L2 weights for the next LC11 neuron
    L2_weights = {L2_id: 0 for L2_id in L2_right_ids}

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from tqdm import tqdm
import warnings
from matplotlib.colors import LinearSegmentedColormap, Normalize
import math

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Replaceable neuron type name
neuron_type = 'LC11'  # You can change it to any neuron type, for example, 'LC3'

# Define paths dynamically using the neuron type
output_dir = f'./result/{neuron_type}/L2_to_{neuron_type}/'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Paths to data files
classification_file_path = './data/classification.txt'
connections_file_path = './data/connections.txt'
synapses_file_path = './data/synapses.txt'
column_assignment_file_path = './data/column_assignment.txt'

# Load data
classification = pd.read_csv(classification_file_path)
connections = pd.read_csv(connections_file_path)
synapses = pd.read_csv(synapses_file_path)

# Get the list of L2 and specific neuron type IDs on the right side
L2_right_ids = classification[(classification['cell_type'] == 'L2') & (classification['side'] == 'right')]['root_id'].tolist()
LC_right_ids = classification[(classification['hemibrain_type'] == neuron_type) & (classification['side'] == 'right')]['root_id'].tolist()

# Load the column assignments
df = pd.read_csv(column_assignment_file_path, dtype={'root_id': str})

# Function to get the 'x' and 'y' coordinates for a given root_id
def get_p_q_by_root_id(root_id):
    result = df[df['root_id'] == root_id]
    if not result.empty:
        return result[['x', 'y']].values[0]
    else:
        return None

# Function to convert hexagonal coordinates to Cartesian coordinates
def hex_to_cartesian(p, q):
    if q % 2 == 1:
        x = 2 * p + 1
    else:
        x = 2 * p
    y = q / 2
    return x, y

# Function to create hexagonal markers for plotting
def create_hexagon_marker(size=1, orientation=0):
    angles = np.linspace(0, 2 * np.pi, 6, endpoint=False) + np.radians(orientation)
    return np.column_stack([np.cos(angles), np.sin(angles)]) * size

# Function to determine the grid size dynamically based on the number of neurons
def determine_subplot_grid(n):
    cols = math.ceil(np.sqrt(n))
    rows = math.ceil(n / cols)
    return rows, cols

# Get the number of neurons for determining the grid size
n_neurons = len(LC_right_ids)

# Determine the layout of subplots
rows, cols = determine_subplot_grid(n_neurons)

# Set the size of each subplot
figsize_per_subplot = (1.5, 2)  # Size for each subplot
fig_width = figsize_per_subplot[0] * cols
fig_height = figsize_per_subplot[1] * rows

# Create subplots with the calculated size
fig, axs = plt.subplots(rows, cols, figsize=(fig_width, fig_height))

# Process each neuron and plot its data onto individual subplots
for i, LC_neuron in tqdm(enumerate(LC_right_ids), desc=f"Processing {neuron_type} Neurons", total=n_neurons):
    all_xy_coords = {tuple(hex_to_cartesian(p, q)): 0 for p, q in df[['x', 'y']].dropna().values}
    file_path = os.path.join(output_dir, f'{LC_neuron}.txt')
    x_coords = []
    y_coords = []
    nag_coords = []

    # Read data from the file and update coordinates with values
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            for line in file:
                parts = line.strip().split(': ')
                if len(parts) == 2:
                    neuron_id = parts[0]
                    if neuron_id in df['root_id'].values:
                        value = float(parts[1])  # Convert the value to float
                        coordinates = get_p_q_by_root_id(neuron_id)
                        if coordinates is not None:
                            x_cartesian, y_cartesian = hex_to_cartesian(coordinates[0], coordinates[1])  # Convert to Cartesian coordinates
                            all_xy_coords[(x_cartesian, y_cartesian)] = value  # Update value
                            x_coords.append(x_cartesian)
                            y_coords.append(y_cartesian)
                            if value < 0:
                                nag_coords.append((x_cartesian, y_cartesian))

    # Get the current subplot
    ax = axs[i // cols, i % cols]

    # Set the background color to white for the subplot
    ax.set_facecolor('white')

    hexagon_marker = create_hexagon_marker(size=1, orientation=0)  # Create rotated hexagon marker

    # Get the maximum positive and negative values, defaulting to 0 if not present
    max_positive = max((v for v in all_xy_coords.values() if v > 0), default=0)
    max_negative = abs(min((v for v in all_xy_coords.values() if v < 0), default=0))
    max_val = max(max_positive, max_negative)

    # Define color map from blue to red
    cmap = LinearSegmentedColormap.from_list('blue_red', ['blue', 'white', 'red'])

    # Normalize the values
    norm = Normalize(vmin=-max_val, vmax=max_val)

    # Function to determine the color based on value
    def get_color(value, max_val):
        alpha = max(abs(value) / max_val, 0.1)  # Adjust alpha based on the value, with a minimum of 0.1
        if value > 0:
            return (1, 0, 0, alpha)  # Red for positive values
        elif value < 0:
            return (0, 0, 1, alpha)  # Blue for negative values
        else:
            return (0, 0, 0, 1)  # White for zero values

    # Assign colors to each coordinate based on its value
    colors = [get_color(all_xy_coords[(x, y)], max_val) for x, y in zip(x_coords, y_coords)]

    # Plot the scatter plot for each neuron
    if x_coords:  # Only plot if there is data
        ax.scatter(x_coords, y_coords, c=colors, s=10, marker=hexagon_marker)
    
    # Set the title to the neuron ID with the appropriate font and size
    ax.set_title(f'{LC_neuron}', fontsize=10, fontname='Times New Roman')
    
    # Hide axis and grid for clean presentation
    ax.set_axis_off()
    
# Remove extra empty subplots and hide their axes
for i in range(len(LC_right_ids), rows * cols):
    fig.delaxes(axs.flat[i])  # Remove the extra subplot frames

# Set the main title for the entire figure with the specified font and size
plt.suptitle(f"Weights of L2 Neurons Corresponding to Neuron Type: {neuron_type}", fontsize=20, fontname='Times New Roman')

# Adjust the layout of the subplots to fit the figure
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Leave space for the main title
plt.show()
